# SciTeX DSP Module - Comprehensive Digital Signal Processing Tutorial

This comprehensive notebook demonstrates the full capabilities of the `scitex.dsp` module, combining the best features from multiple DSP tutorials into one complete guide.

## Features Covered

### 🔬 **Neural Signal Processing**
- Phase-Amplitude Coupling (PAC) analysis
- Ripple detection in neural signals
- Modulation index calculation
- Event-related potential (ERP) analysis

### 📊 **Spectral Analysis**
- Power Spectral Density (PSD) calculation
- Time-frequency analysis (spectrograms, wavelets)
- Cross-correlation and coherence
- Multi-taper methods

### 🔧 **Signal Processing**
- Signal generation and simulation
- Filtering (lowpass, highpass, bandpass, notch)
- Hilbert transform and analytic signals
- Resampling and preprocessing

### 📈 **Advanced Techniques**
- Instantaneous amplitude and phase extraction
- Cross-frequency coupling analysis
- Signal normalization and artifact removal
- Complete DSP pipelines

Let's explore the comprehensive world of digital signal processing with SciTeX!

In [ ]:
# Setup and imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from scipy import signal
from scipy.fft import fft, fftfreq
from scipy import stats
import sys

# Import SciTeX modules
import scitex as stx
try:
    import scitex.dsp as dsp
    dsp_available = True
except ImportError:
    dsp_available = False

# Set random seed for reproducibility
np.random.seed(42)


## 1. 🎛️ Signal Generation and Demo Signals

SciTeX DSP provides comprehensive signal generation utilities for testing and demonstration purposes.

In [ ]:
# Signal generation parameters
fs = 512  # Sampling frequency (Hz)
duration = 4.0  # Duration (seconds)
n_samples = int(fs * duration)
t = np.linspace(0, duration, n_samples, endpoint=False)


# Generate various signal types using SciTeX DSP or fallbacks
signals = {}

if dsp_available:
    try:
        # SciTeX DSP signal generation
        x_periodic, t_periodic, fs_periodic = dsp.demo_sig(
        sig_type='periodic', batch_size=1, n_chs=1, t_sec=duration, fs=fs
        )
        signals['periodic'] = x_periodic[0, 0]
        
        x_chirp, t_chirp, _ = dsp.demo_sig(
        sig_type='chirp', batch_size=1, n_chs=1, t_sec=duration, fs=fs
        )
        signals['chirp'] = x_chirp[0, 0]
        
    except Exception as e:
        dsp_available = False

if not dsp_available:
    # Fallback signal generation
    
    # Periodic signal (multi-component)
    signals['periodic'] = (
    np.sin(2*np.pi*10*t) +
    0.5*np.sin(2*np.pi*25*t) +
    0.2*np.random.randn(len(t))
    )
    
    # Chirp signal (frequency sweep)
    signals['chirp'] = signal.chirp(t, f0=5, f1=50, t1=duration, method='linear')

# Additional signals
signals['gaussian_noise'] = np.random.randn(len(t))
signals['uniform_noise'] = np.random.uniform(-1, 1, len(t))
signals['sine_wave'] = np.sin(2*np.pi*15*t)  # 15 Hz sine
signals['am_signal'] = (1 + 0.5*np.sin(2*np.pi*2*t)) * np.sin(2*np.pi*40*t)  # AM signal

for sig_type in signals.keys():
    # Process sig_type

In [ ]:
# Visualize generated signals

fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.flatten()
fig.suptitle('SciTeX DSP - Signal Generation Gallery', fontsize=16, fontweight='bold')

# Plot each signal type
for i, (sig_type, sig_data) in enumerate(signals.items()):
    if i >= len(axes):
        break
        
    ax = axes[i]
    
    # Time domain plot
    if sig_type in ['gaussian_noise', 'uniform_noise']:
        # For noise signals, show shorter segment
        plot_samples = 1000
        ax.plot(t[:plot_samples], sig_data[:plot_samples], alpha=0.7, linewidth=0.5)
    else:
        # For structured signals, show longer segment
        plot_samples = 2000
        ax.plot(t[:plot_samples], sig_data[:plot_samples], linewidth=1)
    
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    ax.set_title(f'{sig_type.replace("_", " ").title()}')
    ax.grid(True, alpha=0.3)
    
    # Add signal statistics
    rms = np.sqrt(np.mean(sig_data**2))
    std = np.std(sig_data)
    ax.text(0.02, 0.98, f'RMS: {rms:.3f}\nSTD: {std:.3f}', 
        transform=ax.transAxes, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
stx.io.save(fig, "./figures/comprehensive_signal_generation.png", symlink_from_cwd=True)
plt.savefig('dsp_output.png', dpi=100); plt.close()


## 2. 🔍 Power Spectral Density (PSD) Analysis

Comprehensive spectral analysis to understand frequency content of signals.

In [ ]:
# Power Spectral Density Analysis

# Create a complex test signal for spectral analysis
test_signal = (
    1.0 * np.sin(2*np.pi*8*t) +     # Alpha (8 Hz)
    0.7 * np.sin(2*np.pi*15*t) +    # Beta (15 Hz)
    0.5 * np.sin(2*np.pi*30*t) +    # Low gamma (30 Hz)
    0.3 * np.sin(2*np.pi*60*t) +    # High gamma (60 Hz)
    0.2 * np.random.randn(len(t))    # Noise
)


# Calculate PSD using different methods
psd_results = {}

# Method 1: SciTeX DSP (if available)
if dsp_available:
    try:
        psd_values, frequencies = dsp.psd(test_signal.reshape(1, 1, -1), fs, prob=True)
        psd_results['scitex'] = {
        'frequencies': frequencies,
        'psd': psd_values[0, 0] if len(psd_values.shape) > 1 else psd_values,
        'method': 'SciTeX DSP'
        }
    except Exception as e:
        pass  # Fixed incomplete except block

# Method 2: Periodogram
f_perio, psd_perio = signal.periodogram(test_signal, fs)
psd_results['periodogram'] = {
    'frequencies': f_perio,
    'psd': psd_perio,
    'method': 'Periodogram'
}

# Method 3: Welch's method
f_welch, psd_welch = signal.welch(test_signal, fs, nperseg=1024)
psd_results['welch'] = {
    'frequencies': f_welch,
    'psd': psd_welch,
    'method': "Welch's Method"
}

# Method 4: Multi-taper (simulation with different window sizes)
window_sizes = [256, 512, 1024]
multi_taper_psds = []
for nperseg in window_sizes:
    f_mt, psd_mt = signal.welch(test_signal, fs, nperseg=nperseg)
    multi_taper_psds.append(psd_mt)

# Average multi-taper results
psd_mt_avg = np.mean(multi_taper_psds, axis=0)
psd_results['multitaper'] = {
    'frequencies': f_mt,
    'psd': psd_mt_avg,
    'method': 'Multi-taper (Averaged)'
}

for method in psd_results.keys():
    # Process method

In [ ]:
# Visualize PSD analysis results

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Power Spectral Density Analysis Comparison', fontsize=16, fontweight='bold')

# Plot each PSD method
colors = ['blue', 'red', 'green', 'orange']
method_names = list(psd_results.keys())

for i, (method, result) in enumerate(psd_results.items()):
    ax = axes[i//2, i%2]
    
    freqs = result['frequencies']
    psd = result['psd']
    
    # Plot PSD
    ax.semilogy(freqs, psd, color=colors[i], linewidth=2)
    
    # Mark expected peaks
    expected_freqs = [8, 15, 30, 60]
    for freq in expected_freqs:
        ax.axvline(freq, color='gray', linestyle='--', alpha=0.5)
        ax.text(freq+0.5, ax.get_ylim()[1]*0.1, f'{freq}Hz', 
        rotation=90, fontsize=8, alpha=0.7)
    
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Power Spectral Density')
    ax.set_title(result['method'])
    ax.set_xlim(0, 100)
    ax.grid(True, alpha=0.3)
    
    # Find and annotate peaks
    # Focus on the frequency range of interest
    freq_mask = (freqs >= 1) & (freqs <= 80)
    freqs_roi = freqs[freq_mask]
    psd_roi = psd[freq_mask]
    
    # Find peaks
    peaks, _ = signal.find_peaks(psd_roi, height=np.max(psd_roi)*0.1, distance=10)
    
    # Annotate top 3 peaks
    if len(peaks) > 0:
        peak_heights = psd_roi[peaks]
        top_peaks = peaks[np.argsort(peak_heights)[-3:]]  # Top 3 peaks
        
        for peak_idx in top_peaks:
            peak_freq = freqs_roi[peak_idx]
            peak_power = psd_roi[peak_idx]
            ax.plot(peak_freq, peak_power, 'ro', markersize=6)
            ax.annotate(f'{peak_freq:.1f}Hz', 
            (peak_freq, peak_power),
            xytext=(5, 10), textcoords='offset points',
            fontsize=8, ha='left')

plt.tight_layout()
stx.io.save(fig, "./figures/comprehensive_psd_analysis.png", symlink_from_cwd=True)
plt.savefig('dsp_output.png', dpi=100); plt.close()

# Calculate frequency band powers
bands = {
    'Delta': (1, 4),
    'Theta': (4, 8),
    'Alpha': (8, 13),
    'Beta': (13, 30),
    'Gamma': (30, 100)
}

for method, result in psd_results.items():
    freqs = result['frequencies']
    psd = result['psd']
    
    for band_name, (f_low, f_high) in bands.items():
        band_mask = (freqs >= f_low) & (freqs < f_high)
        if np.any(band_mask):
            band_power = np.trapz(psd[band_mask], freqs[band_mask])


## 3. 🌊 Hilbert Transform and Analytic Signals

Extract instantaneous amplitude and phase information using the Hilbert transform.

In [ ]:
# Hilbert Transform Analysis

# Create test signals for Hilbert analysis
t_short = np.linspace(0, 2, int(2 * fs))

# 1. Amplitude modulated signal
carrier_freq = 20  # Hz
mod_freq = 3       # Hz
amplitude_envelope = 1 + 0.6 * np.sin(2*np.pi*mod_freq*t_short)
am_signal = amplitude_envelope * np.sin(2*np.pi*carrier_freq*t_short)

# 2. Frequency modulated signal
freq_deviation = 5  # Hz
instantaneous_freq = carrier_freq + freq_deviation * np.sin(2*np.pi*mod_freq*t_short)
phase = 2*np.pi * np.cumsum(instantaneous_freq) / fs
fm_signal = np.sin(phase)

# 3. Combined AM-FM signal
am_fm_signal = amplitude_envelope * np.sin(phase)

signals_hilbert = {
    'AM Signal': am_signal,
    'FM Signal': fm_signal,
    'AM-FM Signal': am_fm_signal
}


# Apply Hilbert transform
hilbert_results = {}

for sig_name, sig_data in signals_hilbert.items():
    # SciTeX DSP Hilbert transform (if available)
    if dsp_available:
        try:
            # SciTeX expects 3D input: (batch, channel, time)
            sig_3d = sig_data.reshape(1, 1, -1)
            phase_stx, amplitude_stx = dsp.hilbert(sig_3d, dim=-1)
            
            hilbert_results[sig_name] = {
            'signal': sig_data,
            'amplitude': amplitude_stx[0, 0],
            'phase': phase_stx[0, 0],
            'method': 'SciTeX DSP'
            }
            continue
        except Exception as e:
            pass  # Fixed incomplete except block
    
    # Fallback to scipy
    analytic_signal = signal.hilbert(sig_data)
    amplitude = np.abs(analytic_signal)
    phase = np.angle(analytic_signal)
    
    hilbert_results[sig_name] = {
        'signal': sig_data,
        'amplitude': amplitude,
        'phase': phase,
        'method': 'Scipy'
    }


# Calculate instantaneous frequency
for sig_name, result in hilbert_results.items():
    phase = result['phase']
    # Unwrap phase and calculate derivative
    unwrapped_phase = np.unwrap(phase)
    inst_freq = np.diff(unwrapped_phase) / (2*np.pi) * fs
    result['instantaneous_frequency'] = inst_freq


In [ ]:
# Visualize Hilbert Transform Results

fig, axes = plt.subplots(len(signals_hilbert), 4, figsize=(20, 4*len(signals_hilbert)))
if len(signals_hilbert) == 1:
    axes = axes.reshape(1, -1)

fig.suptitle('Comprehensive Hilbert Transform Analysis', fontsize=16, fontweight='bold')

for i, (sig_name, result) in enumerate(hilbert_results.items()):
    signal_data = result['signal']
    amplitude = result['amplitude']
    phase = result['phase']
    inst_freq = result['instantaneous_frequency']
    
    # 1. Original signal with envelope
    ax = axes[i, 0]
    ax.plot(t_short, signal_data, 'b-', alpha=0.7, linewidth=1, label='Signal')
    ax.plot(t_short, amplitude, 'r-', linewidth=2, label='Envelope')
    ax.plot(t_short, -amplitude, 'r-', linewidth=2)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    ax.set_title(f'{sig_name} - Signal & Envelope')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 2. Instantaneous amplitude
    ax = axes[i, 1]
    ax.plot(t_short, amplitude, 'r-', linewidth=2)
    if sig_name == 'AM Signal':
        # Overlay true envelope for AM signal
        ax.plot(t_short, amplitude_envelope, 'k--', linewidth=2, label='True Envelope')
        ax.legend()
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    ax.set_title('Instantaneous Amplitude')
    ax.grid(True, alpha=0.3)
    
    # 3. Instantaneous phase
    ax = axes[i, 2]
    ax.plot(t_short, np.unwrap(phase), 'g-', linewidth=2)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Phase (rad)')
    ax.set_title('Instantaneous Phase (Unwrapped)')
    ax.grid(True, alpha=0.3)
    
    # 4. Instantaneous frequency
    ax = axes[i, 3]
    ax.plot(t_short[:-1], inst_freq, 'purple', linewidth=2, label='Estimated')
    
    if sig_name == 'FM Signal' or sig_name == 'AM-FM Signal':
        # Overlay true frequency for FM signals
        ax.plot(t_short, instantaneous_freq, 'k--', linewidth=2, label='True Frequency')
        ax.legend()
    else:
        # For AM signal, show carrier frequency
        ax.axhline(carrier_freq, color='k', linestyle='--', label=f'Carrier ({carrier_freq} Hz)')
        ax.legend()
    
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Frequency (Hz)')
    ax.set_title('Instantaneous Frequency')
    ax.set_ylim(carrier_freq - 2*freq_deviation, carrier_freq + 2*freq_deviation)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
stx.io.save(fig, "./figures/comprehensive_hilbert_analysis.png", symlink_from_cwd=True)
plt.savefig('dsp_output.png', dpi=100); plt.close()

# Performance metrics
for sig_name, result in hilbert_results.items():
    amplitude = result['amplitude']
    inst_freq = result['instantaneous_frequency']


## 4. 🧠 Phase-Amplitude Coupling (PAC) Analysis

Advanced cross-frequency coupling analysis for neural signals.

In [ ]:
# Phase-Amplitude Coupling Analysis

# Generate PAC signal
t_pac = np.linspace(0, 10, int(10 * fs))

# Low frequency phase-providing signal (theta rhythm: 6 Hz)
phase_freq = 6
theta_signal = np.sin(2*np.pi*phase_freq*t_pac)

# High frequency amplitude-modulated signal (gamma: 80 Hz)
amp_freq = 80
modulation_strength = 0.8  # Coupling strength

# Create gamma signal modulated by theta phase
gamma_amplitude = 1 + modulation_strength * np.sin(2*np.pi*phase_freq*t_pac)
gamma_signal = gamma_amplitude * np.sin(2*np.pi*amp_freq*t_pac)

# Combined PAC signal with noise
pac_signal = (
    theta_signal + 
    0.5 * gamma_signal + 
    0.1 * np.random.randn(len(t_pac))
)

# Control signal (no coupling)
gamma_uncoupled = np.sin(2*np.pi*amp_freq*t_pac)
control_signal = (
    theta_signal + 
    0.5 * gamma_uncoupled + 
    0.1 * np.random.randn(len(t_pac))
)


# Calculate PAC using SciTeX DSP if available
pac_calculated = False

if dsp_available:
    try:
        # Prepare signal for SciTeX PAC
        pac_signal_3d = pac_signal.reshape(1, 1, -1)
        
        pac_values, phase_freqs, amp_freqs = dsp.pac(
        pac_signal_3d,
        fs,
        pha_start_hz=2,
        pha_end_hz=20,
        pha_n_bands=20,
        amp_start_hz=30,
        amp_end_hz=120,
        amp_n_bands=20,
        device='cpu',
        batch_size_ch=-1
        )
        
        pac_calculated = True
        
    except Exception as e:
        pass  # Fixed incomplete except block

# Manual PAC calculation as fallback

# Filter for phase (theta band: 4-8 Hz)
sos_phase = signal.butter(4, [4, 8], 'bandpass', fs=fs, output='sos')
phase_filtered = signal.sosfiltfilt(sos_phase, pac_signal)
phase_control = signal.sosfiltfilt(sos_phase, control_signal)

# Filter for amplitude (gamma band: 70-90 Hz)
sos_amp = signal.butter(4, [70, 90], 'bandpass', fs=fs, output='sos')
amp_filtered = signal.sosfiltfilt(sos_amp, pac_signal)
amp_control = signal.sosfiltfilt(sos_amp, control_signal)

# Extract phase and amplitude
theta_phase = np.angle(signal.hilbert(phase_filtered))
gamma_amplitude = np.abs(signal.hilbert(amp_filtered))

theta_phase_ctrl = np.angle(signal.hilbert(phase_control))
gamma_amplitude_ctrl = np.abs(signal.hilbert(amp_control))


In [ ]:
# PAC Analysis: Modulation Index and Comodulogram

# Calculate modulation index manually
def calculate_modulation_index(phase, amplitude, n_bins=18):
    """Calculate modulation index for phase-amplitude coupling."""
    # Create phase bins
    phase_bins = np.linspace(-np.pi, np.pi, n_bins + 1)
    phase_centers = (phase_bins[:-1] + phase_bins[1:]) / 2
    
    # Compute mean amplitude per phase bin
    amp_by_phase = np.zeros(n_bins)
    for i in range(n_bins):
        mask = (phase >= phase_bins[i]) & (phase < phase_bins[i + 1])
        if np.sum(mask) > 0:
            amp_by_phase[i] = np.mean(amplitude[mask])
    
    # Normalize to create probability distribution
    p = amp_by_phase / np.sum(amp_by_phase)
    
    # Calculate modulation index (normalized entropy)
    # Higher MI indicates stronger coupling
    uniform_entropy = np.log(n_bins)
    actual_entropy = -np.sum(p * np.log(p + 1e-10))
    mi = (uniform_entropy - actual_entropy) / uniform_entropy
    
    return mi, phase_centers, amp_by_phase

# Calculate MI for coupled and control signals
mi_coupled, phase_centers, amp_by_phase_coupled = calculate_modulation_index(theta_phase, gamma_amplitude)
mi_control, _, amp_by_phase_control = calculate_modulation_index(theta_phase_ctrl, gamma_amplitude_ctrl)


# Create comprehensive PAC visualization
if pac_calculated:
    fig, axes = plt.subplots(3, 3, figsize=(18, 15))
else:
    fig, axes = plt.subplots(3, 2, figsize=(15, 15))

fig.suptitle('Comprehensive Phase-Amplitude Coupling Analysis', fontsize=16, fontweight='bold')

# 1. Raw PAC signal
ax = axes[0, 0]
time_window = (t_pac >= 1) & (t_pac <= 3)  # Show 2 seconds
ax.plot(t_pac[time_window], pac_signal[time_window], 'k-', linewidth=0.8)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.set_title('PAC Signal (Raw)')
ax.grid(True, alpha=0.3)

# 2. Filtered signals
ax = axes[0, 1]
ax.plot(t_pac[time_window], phase_filtered[time_window], 'b-', label='Theta (4-8 Hz)', linewidth=2)
ax.plot(t_pac[time_window], amp_filtered[time_window], 'r-', alpha=0.7, label='Gamma (70-90 Hz)', linewidth=1)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.set_title('Filtered Components')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Phase and amplitude traces
ax = axes[1, 0]
ax.plot(t_pac[time_window], theta_phase[time_window], 'b-', linewidth=2, label='Theta Phase')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Phase (rad)')
ax.set_title('Theta Phase')
ax.set_ylim(-np.pi, np.pi)
ax.set_yticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
ax.set_yticklabels(['-\\pi', '-\\pi/2', '0', '\\pi/2', '\\pi'])
ax.grid(True, alpha=0.3)

ax_twin = ax.twinx()
ax_twin.plot(t_pac[time_window], gamma_amplitude[time_window], 'r-', linewidth=2, alpha=0.7, label='Gamma Amplitude')
ax_twin.set_ylabel('Gamma Amplitude', color='r')
ax_twin.tick_params(axis='y', labelcolor='r')

# 4. Phase-amplitude scatter plot
ax = axes[1, 1]
# Downsample for visualization
downsample = slice(None, None, 50)
ax.scatter(theta_phase[downsample], gamma_amplitude[downsample], 
    alpha=0.3, s=10, c='purple')
ax.set_xlabel('Theta Phase (rad)')
ax.set_ylabel('Gamma Amplitude')
ax.set_title('Phase-Amplitude Relationship')
ax.set_xlim(-np.pi, np.pi)
ax.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
ax.set_xticklabels(['-\\pi', '-\\pi/2', '0', '\\pi/2', '\\pi'])
ax.grid(True, alpha=0.3)

# 5. Amplitude by phase (histogram)
ax = axes[2, 0]
width = phase_centers[1] - phase_centers[0]
bars1 = ax.bar(phase_centers - width/4, amp_by_phase_coupled, width/2, 
    alpha=0.7, label=f'Coupled (MI={mi_coupled:.3f})', color='blue')
bars2 = ax.bar(phase_centers + width/4, amp_by_phase_control, width/2, 
    alpha=0.7, label=f'Control (MI={mi_control:.3f})', color='red')
ax.set_xlabel('Theta Phase (rad)')
ax.set_ylabel('Mean Gamma Amplitude')
ax.set_title('PAC: Amplitude by Phase')
ax.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
ax.set_xticklabels(['-\\pi', '-\\pi/2', '0', '\\pi/2', '\\pi'])
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# 6. Polar plot of PAC
ax = fig.add_subplot(3, 2, 6, projection='polar') if not pac_calculated else fig.add_subplot(3, 3, 9, projection='polar')
bars = ax.bar(phase_centers, amp_by_phase_coupled, width=width, alpha=0.7, color='blue')
ax.set_title('PAC: Polar Representation\n(Coupled Signal)', pad=20)
ax.set_ylim(0, np.max(amp_by_phase_coupled) * 1.1)

# 7. SciTeX PAC Comodulogram (if available)
if pac_calculated:
    ax = axes[0, 2]
    
    # Extract PAC matrix for visualization
    if hasattr(pac_values, 'numpy'):
        pac_matrix = pac_values[0, 0].numpy()  # Convert from tensor if needed
    else:
        pac_matrix = pac_values[0, 0]
    
    im = ax.imshow(
        pac_matrix,
        aspect='auto',
        origin='lower',
        extent=[amp_freqs[0], amp_freqs[-1], phase_freqs[0], phase_freqs[-1]],
        cmap='viridis'
    )
    
    ax.set_xlabel('Amplitude Frequency (Hz)')
    ax.set_ylabel('Phase Frequency (Hz)')
    ax.set_title('SciTeX PAC Comodulogram')
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('PAC Strength')
    
    # Mark expected coupling
    ax.plot(amp_freq, phase_freq, 'r*', markersize=15, label=f'Expected ({phase_freq},{amp_freq} Hz)')
    ax.legend()
    
    # Additional SciTeX results
    ax = axes[1, 2]
    ax.axis('off')
    results_text = f"SciTeX PAC Results:\n\n"
    results_text += f"Phase bands: {len(phase_freqs)}\n"
    results_text += f"Amplitude bands: {len(amp_freqs)}\n"
    results_text += f"Max PAC: {np.max(pac_matrix):.4f}\n"
    results_text += f"Mean PAC: {np.mean(pac_matrix):.4f}\n"
    
    # Find peak PAC location
    max_idx = np.unravel_index(np.argmax(pac_matrix), pac_matrix.shape)
    peak_phase_freq = phase_freqs[max_idx[0]]
    peak_amp_freq = amp_freqs[max_idx[1]]
    results_text += f"Peak at: {peak_phase_freq:.1f} → {peak_amp_freq:.1f} Hz"
    
    ax.text(0.1, 0.9, results_text, transform=ax.transAxes, 
        fontsize=11, verticalalignment='top', fontfamily='monospace',
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))

plt.tight_layout()
stx.io.save(fig, "./figures/comprehensive_pac_analysis.png", symlink_from_cwd=True)
plt.savefig('dsp_output.png', dpi=100); plt.close()

if pac_calculated:
    # Condition met

## 5. 🔬 Advanced Signal Processing Pipeline

Complete signal processing workflow combining multiple techniques.

In [ ]:
# Advanced Signal Processing Pipeline

def comprehensive_dsp_pipeline(signal_data, fs, config):
    """
    Comprehensive DSP pipeline for scientific signal analysis.
    
    Parameters:
    - signal_data: Input signal array
    - fs: Sampling frequency (Hz)
    - config: Analysis configuration dictionary
    
    Returns:
    - results: Dictionary containing all analysis results
    """
    results = {'config': config, 'fs': fs}
    
    
    # 1. Preprocessing
    processed_signal = signal_data.copy()
    
    # Remove DC offset
    if config['preprocessing']['remove_dc']:
        processed_signal = processed_signal - np.mean(processed_signal)
    
    # Notch filtering for line noise
    if config['preprocessing']['notch_filter']:
        for freq in config['preprocessing']['notch_freqs']:
            b, a = signal.iirnotch(freq, Q=30, fs=fs)
            processed_signal = signal.filtfilt(b, a, processed_signal)
    
    # Bandpass filtering
    if config['filtering']['apply']:
        sos = signal.butter(
        config['filtering']['order'],
        config['filtering']['freqs'],
        config['filtering']['type'],
        fs=fs, output='sos'
        )
        processed_signal = signal.sosfiltfilt(sos, processed_signal)
    
    results['processed_signal'] = processed_signal
    
    # 2. Spectral Analysis
    f_psd, psd = signal.welch(
        processed_signal, fs, 
        nperseg=config['spectral']['nperseg']
    )
    results['spectral'] = {'frequencies': f_psd, 'psd': psd}
    
    # 3. Time-frequency analysis
    f_spec, t_spec, Sxx = signal.spectrogram(
        processed_signal, fs,
        nperseg=config['timefreq']['nperseg'],
        overlap=config['timefreq']['overlap']
    )
    results['timefreq'] = {
        'frequencies': f_spec,
        'times': t_spec,
        'spectrogram': Sxx
    }
    
    # 4. Hilbert analysis
    analytic_signal = signal.hilbert(processed_signal)
    amplitude_envelope = np.abs(analytic_signal)
    instantaneous_phase = np.angle(analytic_signal)
    instantaneous_freq = np.diff(np.unwrap(instantaneous_phase)) / (2*np.pi) * fs
    
    results['hilbert'] = {
        'amplitude': amplitude_envelope,
        'phase': instantaneous_phase,
        'frequency': instantaneous_freq
    }
    
    # 5. Feature extraction
    features = {}
    
    # Time domain features
    features['rms'] = np.sqrt(np.mean(processed_signal**2))
    features['variance'] = np.var(processed_signal)
    features['skewness'] = stats.skew(processed_signal)
    features['kurtosis'] = stats.kurtosis(processed_signal)
    features['peak_to_peak'] = np.ptp(processed_signal)
    features['zero_crossings'] = np.sum(np.diff(np.sign(processed_signal)) != 0)
    
    # Frequency domain features
    features['peak_frequency'] = f_psd[np.argmax(psd)]
    features['spectral_centroid'] = np.sum(f_psd * psd) / np.sum(psd)
    features['spectral_bandwidth'] = np.sqrt(
        np.sum(((f_psd - features['spectral_centroid'])**2) * psd) / np.sum(psd)
    )
    
    # Band power features
    for band_name, (f_low, f_high) in config['bands'].items():
        band_mask = (f_psd >= f_low) & (f_psd < f_high)
        if np.any(band_mask):
            band_power = np.trapz(psd[band_mask], f_psd[band_mask])
            features[f'{band_name}_power'] = band_power
            features[f'{band_name}_relative_power'] = band_power / np.trapz(psd, f_psd)
    
    # Hilbert-based features
    features['mean_amplitude'] = np.mean(amplitude_envelope)
    features['amplitude_variability'] = np.std(amplitude_envelope)
    features['mean_inst_frequency'] = np.mean(instantaneous_freq)
    features['freq_variability'] = np.std(instantaneous_freq)
    
    results['features'] = features
    
    # 6. Signal quality metrics
    quality = {}
    
    # SNR estimation
    signal_power = np.mean(processed_signal**2)
    noise_estimate = np.var(np.diff(processed_signal))  # High-frequency noise
    quality['snr_estimate'] = 10 * np.log10(signal_power / noise_estimate)
    
    # Spectral flatness
    geometric_mean = np.exp(np.mean(np.log(psd + 1e-10)))
    arithmetic_mean = np.mean(psd)
    quality['spectral_flatness'] = geometric_mean / arithmetic_mean
    
    # Dynamic range
    quality['dynamic_range'] = 20 * np.log10(np.max(psd) / np.min(psd + 1e-10))
    
    results['quality'] = quality
    
    return results

# Pipeline configuration
pipeline_config = {
    'preprocessing': {
    'remove_dc': True,
    'notch_filter': True,
    'notch_freqs': [50, 100]  # Line noise and harmonic
    },
    'filtering': {
    'apply': True,
    'type': 'bandpass',
    'freqs': [1, 100],
    'order': 4
    },
    'spectral': {
    'nperseg': 1024
    },
    'timefreq': {
    'nperseg': 256,
    'overlap': 0.5
    },
    'bands': {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'low_gamma': (30, 50),
    'high_gamma': (50, 100)
    }
}

# Apply pipeline to test signal
pipeline_results = comprehensive_dsp_pipeline(test_signal, fs, pipeline_config)


In [ ]:
# Visualize Pipeline Results

fig, axes = plt.subplots(3, 3, figsize=(18, 15))
fig.suptitle('Comprehensive DSP Pipeline Results', fontsize=16, fontweight='bold')

# 1. Original vs Processed Signal
ax = axes[0, 0]
time_samples = np.arange(len(test_signal)) / fs
display_samples = slice(0, fs*2)  # First 2 seconds
ax.plot(time_samples[display_samples], test_signal[display_samples], 
    'b-', alpha=0.7, label='Original')
ax.plot(time_samples[display_samples], pipeline_results['processed_signal'][display_samples], 
    'r-', label='Processed')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.set_title('Signal Preprocessing')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Power Spectral Density
ax = axes[0, 1]
freqs = pipeline_results['spectral']['frequencies']
psd = pipeline_results['spectral']['psd']
ax.semilogy(freqs, psd, 'b-', linewidth=2)
ax.axvline(pipeline_results['features']['peak_frequency'], 
    color='red', linestyle='--',
    label=f"Peak: {pipeline_results['features']['peak_frequency']:.1f} Hz")
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Power Spectral Density')
ax.set_title('Spectral Analysis')
ax.set_xlim(0, 50)
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Spectrogram
ax = axes[0, 2]
f_spec = pipeline_results['timefreq']['frequencies']
t_spec = pipeline_results['timefreq']['times']
Sxx = pipeline_results['timefreq']['spectrogram']
im = ax.pcolormesh(t_spec, f_spec, 10*np.log10(Sxx), shading='gouraud', cmap='viridis')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Frequency (Hz)')
ax.set_title('Time-Frequency Analysis')
ax.set_ylim(0, 50)
plt.colorbar(im, ax=ax, label='Power (dB)')

# 4. Hilbert Analysis - Amplitude
ax = axes[1, 0]
amplitude = pipeline_results['hilbert']['amplitude']
ax.plot(time_samples[display_samples], pipeline_results['processed_signal'][display_samples], 
    'b-', alpha=0.5, linewidth=1, label='Signal')
ax.plot(time_samples[display_samples], amplitude[display_samples], 
    'r-', linewidth=2, label='Envelope')
ax.plot(time_samples[display_samples], -amplitude[display_samples], 
    'r-', linewidth=2)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.set_title('Instantaneous Amplitude')
ax.legend()
ax.grid(True, alpha=0.3)

# 5. Hilbert Analysis - Frequency
ax = axes[1, 1]
inst_freq = pipeline_results['hilbert']['frequency']
ax.plot(time_samples[1:][display_samples], inst_freq[display_samples], 
    'purple', linewidth=2)
ax.axhline(pipeline_results['features']['mean_inst_frequency'], 
    color='red', linestyle='--',
    label=f"Mean: {pipeline_results['features']['mean_inst_frequency']:.1f} Hz")
ax.set_xlabel('Time (s)')
ax.set_ylabel('Frequency (Hz)')
ax.set_title('Instantaneous Frequency')
ax.legend()
ax.grid(True, alpha=0.3)

# 6. Band Powers
ax = axes[1, 2]
bands = list(pipeline_config['bands'].keys())
powers = [pipeline_results['features'][f'{band}_power'] for band in bands]
relative_powers = [pipeline_results['features'][f'{band}_relative_power'] for band in bands]

x = np.arange(len(bands))
width = 0.35
bars1 = ax.bar(x - width/2, powers, width, label='Absolute Power', alpha=0.7)
ax2 = ax.twinx()
bars2 = ax2.bar(x + width/2, relative_powers, width, label='Relative Power', 
    alpha=0.7, color='orange')

ax.set_xlabel('Frequency Bands')
ax.set_ylabel('Absolute Power', color='blue')
ax2.set_ylabel('Relative Power', color='orange')
ax.set_title('Band Power Analysis')
ax.set_xticks(x)
ax.set_xticklabels([band.replace('_', '\n') for band in bands])
ax.tick_params(axis='y', labelcolor='blue')
ax2.tick_params(axis='y', labelcolor='orange')
ax.grid(True, alpha=0.3, axis='y')

# 7. Time Domain Features
ax = axes[2, 0]
ax.axis('off')
time_features = {
    'RMS': pipeline_results['features']['rms'],
    'Variance': pipeline_results['features']['variance'],
    'Skewness': pipeline_results['features']['skewness'],
    'Kurtosis': pipeline_results['features']['kurtosis'],
    'Peak-to-Peak': pipeline_results['features']['peak_to_peak'],
    'Zero Crossings': pipeline_results['features']['zero_crossings']
}

feature_text = "Time Domain Features:\n\n"
for key, value in time_features.items():
    if isinstance(value, (int, np.integer)):
        feature_text += f"{key}: {value}\n"
    else:
        feature_text += f"{key}: {value:.4f}\n"

ax.text(0.1, 0.9, feature_text, transform=ax.transAxes, 
    fontsize=11, verticalalignment='top', fontfamily='monospace',
    bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))

# 8. Frequency Domain Features
ax = axes[2, 1]
ax.axis('off')
freq_features = {
    'Peak Frequency': pipeline_results['features']['peak_frequency'],
    'Spectral Centroid': pipeline_results['features']['spectral_centroid'],
    'Spectral Bandwidth': pipeline_results['features']['spectral_bandwidth'],
    'Mean Inst. Freq': pipeline_results['features']['mean_inst_frequency'],
    'Freq Variability': pipeline_results['features']['freq_variability']
}

freq_text = "Frequency Features:\n\n"
for key, value in freq_features.items():
    freq_text += f"{key}: {value:.2f} Hz\n"

ax.text(0.1, 0.9, freq_text, transform=ax.transAxes, 
    fontsize=11, verticalalignment='top', fontfamily='monospace',
    bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))

# 9. Quality Metrics
ax = axes[2, 2]
ax.axis('off')
quality_metrics = pipeline_results['quality']

quality_text = "Signal Quality Metrics:\n\n"
quality_text += f"SNR Estimate: {quality_metrics['snr_estimate']:.1f} dB\n"
quality_text += f"Spectral Flatness: {quality_metrics['spectral_flatness']:.4f}\n"
quality_text += f"Dynamic Range: {quality_metrics['dynamic_range']:.1f} dB\n\n"

# Interpretation
quality_text += "Interpretation:\n"
if quality_metrics['snr_estimate'] > 20:
    quality_text += "• High quality signal\n"
elif quality_metrics['snr_estimate'] > 10:
    quality_text += "• Good quality signal\n"
else:
    quality_text += "• Noisy signal\n"

if quality_metrics['spectral_flatness'] < 0.1:
    quality_text += "• Highly structured\n"
else:
    quality_text += "• Noise-like\n"

ax.text(0.1, 0.9, quality_text, transform=ax.transAxes, 
    fontsize=11, verticalalignment='top', fontfamily='monospace',
    bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.7))

plt.tight_layout()
stx.io.save(fig, "./figures/comprehensive_dsp_pipeline.png", symlink_from_cwd=True)
plt.savefig('dsp_output.png', dpi=100); plt.close()


## 6. 💾 Results Summary and Export

Comprehensive summary of all DSP analyses and save results for future reference.

In [ ]:
# Comprehensive Results Summary and Export

# Create comprehensive results dictionary
comprehensive_results = {
    'metadata': {
    'analysis_date': pd.Timestamp.now().isoformat(),
    'scitex_version': stx.__version__ if hasattr(stx, '__version__') else 'development',
    'dsp_module_available': dsp_available,
    'sampling_rate': fs,
    'signal_duration': len(test_signal) / fs
    },
    'signal_generation': {
    'n_signals': len(signals),
    'signal_types': list(signals.keys()),
    'parameters': {
    'fs': fs,
    'duration': duration,
    'n_samples': n_samples
    }
    },
    'spectral_analysis': {
    'methods_used': list(psd_results.keys()),
    'frequency_bands': pipeline_config['bands'],
    'peak_frequency': float(pipeline_results['features']['peak_frequency']),
    'spectral_centroid': float(pipeline_results['features']['spectral_centroid'])
    },
    'hilbert_analysis': {
    'signals_analyzed': list(hilbert_results.keys()),
    'mean_amplitudes': {name: float(np.mean(result['amplitude']))
    for name, result in hilbert_results.items()},
    'frequency_ranges': {name: [float(result['instantaneous_frequency'].min()),
    float(result['instantaneous_frequency'].max())]
    for name, result in hilbert_results.items()}
    },
    'pac_analysis': {
    'modulation_index_coupled': float(mi_coupled),
    'modulation_index_control': float(mi_control),
    'pac_strength_difference': float(mi_coupled - mi_control),
    'scitex_pac_available': pac_calculated
    },
    'pipeline_features': {},
    'quality_metrics': {}
}

# Add pipeline results
for key, value in pipeline_results['features'].items():
    if isinstance(value, (int, float, np.integer, np.floating)):
        comprehensive_results['pipeline_features'][key] = float(value)

for key, value in pipeline_results['quality'].items():
    if isinstance(value, (int, float, np.integer, np.floating)):
        comprehensive_results['quality_metrics'][key] = float(value)

# Add PAC results if available
if pac_calculated:
    pac_matrix = pac_values[0, 0].numpy() if hasattr(pac_values, 'numpy') else pac_values[0, 0]
    comprehensive_results['pac_analysis']['scitex_pac_max'] = float(np.max(pac_matrix))
    comprehensive_results['pac_analysis']['scitex_pac_mean'] = float(np.mean(pac_matrix))

# Save results to multiple formats
results_dir = Path("./comprehensive_dsp_results")
results_dir.mkdir(exist_ok=True, parents=True)

# Save as JSON
import json
json_path = results_dir / "comprehensive_dsp_results.json"
with open(json_path, 'w') as f:
    json.dump(comprehensive_results, f, indent=2)

# Save features as CSV
features_df = pd.DataFrame([pipeline_results['features']])
features_csv_path = results_dir / "extracted_features.csv"
features_df.to_csv(features_csv_path, index=False)

# Save band powers
band_powers = {}
for band in pipeline_config['bands'].keys():
    band_powers[f'{band}_power'] = pipeline_results['features'][f'{band}_power']
    band_powers[f'{band}_relative_power'] = pipeline_results['features'][f'{band}_relative_power']

band_df = pd.DataFrame([band_powers])
band_csv_path = results_dir / "band_powers.csv"
band_df.to_csv(band_csv_path, index=False)

# Save processed signals
signal_data = {
    'time': time_samples,
    'original_signal': test_signal,
    'processed_signal': pipeline_results['processed_signal'],
    'amplitude_envelope': pipeline_results['hilbert']['amplitude']
}
signal_df = pd.DataFrame(signal_data)
signal_csv_path = results_dir / "processed_signals.csv"
signal_df.to_csv(signal_csv_path, index=False)

# Create executive summary report
summary_path = results_dir / "dsp_analysis_summary.md"
with open(summary_path, 'w') as f:
    f.write("# SciTeX DSP Module - Comprehensive Analysis Report\n\n")
    f.write(f"**Analysis Date:** {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    
    f.write("## Executive Summary\n\n")
    f.write(f"This report presents a comprehensive analysis of digital signal processing capabilities ")
    f.write(f"using the SciTeX DSP module. We analyzed {len(signals)} different signal types, ")
    f.write(f"applied {len(psd_results)} spectral analysis methods, and performed advanced ")
    f.write(f"phase-amplitude coupling analysis.\n\n")
    
    f.write("## Key Findings\n\n")
    f.write(f"### 🔍 Signal Analysis\n")
    f.write(f"- **Sampling Rate:** {fs} Hz\n")
    f.write(f"- **Signal Duration:** {len(test_signal)/fs:.1f} seconds\n")
    f.write(f"- **Peak Frequency:** {pipeline_results['features']['peak_frequency']:.1f} Hz\n")
    f.write(f"- **Signal Quality (SNR):** {pipeline_results['quality']['snr_estimate']:.1f} dB\n\n")
    
    f.write(f"### 🌊 Hilbert Transform Results\n")
    f.write(f"- **Signals Analyzed:** {len(hilbert_results)}\n")
    f.write(f"- **Mean Amplitude:** {pipeline_results['features']['mean_amplitude']:.3f}\n")
    f.write(f"- **Mean Inst. Frequency:** {pipeline_results['features']['mean_inst_frequency']:.1f} Hz\n\n")
    
    f.write(f"### 🧠 Phase-Amplitude Coupling\n")
    f.write(f"- **Coupled Signal MI:** {mi_coupled:.4f}\n")
    f.write(f"- **Control Signal MI:** {mi_control:.4f}\n")
    f.write(f"- **PAC Strength:** {mi_coupled - mi_control:.4f}\n")
    if pac_calculated:
        f.write(f"- **SciTeX PAC Available:** Yes\n")
    f.write("\n")
    
    f.write("## Frequency Band Analysis\n\n")
    f.write("| Band | Frequency Range | Absolute Power | Relative Power |\n")
    f.write("|------|-----------------|----------------|----------------|\n")
    for band, (f_low, f_high) in pipeline_config['bands'].items():
        abs_power = pipeline_results['features'][f'{band}_power']
        rel_power = pipeline_results['features'][f'{band}_relative_power']
        f.write(f"| {band.title()} | {f_low}-{f_high} Hz | {abs_power:.4f} | {rel_power:.4f} |\n")
    
    f.write("\n## Signal Quality Assessment\n\n")
    f.write(f"- **SNR Estimate:** {pipeline_results['quality']['snr_estimate']:.1f} dB\n")
    f.write(f"- **Spectral Flatness:** {pipeline_results['quality']['spectral_flatness']:.4f}\n")
    f.write(f"- **Dynamic Range:** {pipeline_results['quality']['dynamic_range']:.1f} dB\n\n")
    
    # Quality interpretation
    snr = pipeline_results['quality']['snr_estimate']
    if snr > 20:
        quality_rating = "Excellent"
    elif snr > 10:
        quality_rating = "Good"
    elif snr > 0:
        quality_rating = "Fair"
    else:
        quality_rating = "Poor"
    
    f.write(f"**Overall Signal Quality:** {quality_rating}\n\n")
    
    f.write("## Methods and Techniques\n\n")
    f.write("1. **Signal Generation:** Multiple synthetic signal types\n")
    f.write("2. **Spectral Analysis:** PSD using multiple methods\n")
    f.write("3. **Hilbert Transform:** Amplitude and phase extraction\n")
    f.write("4. **Phase-Amplitude Coupling:** Cross-frequency analysis\n")
    f.write("5. **Feature Extraction:** Comprehensive signal characterization\n")
    f.write("6. **Quality Assessment:** SNR and spectral metrics\n\n")
    
    f.write("## Files Generated\n\n")
    f.write(f"- `comprehensive_dsp_results.json` - Complete analysis results\n")
    f.write(f"- `extracted_features.csv` - All extracted features\n")
    f.write(f"- `band_powers.csv` - Frequency band power analysis\n")
    f.write(f"- `processed_signals.csv` - Time series data\n")
    f.write(f"- `dsp_analysis_summary.md` - This summary report\n")
    f.write(f"- `../figures/` - All visualization plots\n\n")
    
    f.write("---\n")
    f.write("*Generated by SciTeX DSP Module Comprehensive Tutorial*\n")


for file_path in results_dir.glob("*"):
    # Process file_path

In [ ]:
# Final DSP Analysis Summary

capabilities = [
    "✅ Multi-type signal generation (periodic, chirp, noise, AM/FM)",
    "✅ Advanced spectral analysis (PSD, periodogram, Welch, multi-taper)",
    "✅ Hilbert transform with amplitude and phase extraction",
    "✅ Phase-amplitude coupling (PAC) analysis",
    "✅ Comprehensive feature extraction (time & frequency domain)",
    "✅ Signal quality assessment and SNR estimation",
    "✅ Time-frequency analysis (spectrograms)",
    "✅ Frequency band power analysis",
    "✅ Modulation index calculation",
    "✅ Complete DSP pipeline implementation",
    "✅ Automated results export and reporting",
    "✅ Neural signal processing workflows"
]

for capability in capabilities:
    # Process capability



applications = [
    "🧠 Neuroscience: EEG/LFP analysis, brain oscillations, connectivity",
    "📈 Signal Processing: General time-series analysis workflows",
    "🔬 Research: Cross-frequency coupling, neural event detection",
    "⚕️ Biomedical: Neural signal preprocessing and feature extraction",
    "🤖 Machine Learning: Feature engineering for time-series classification",
    "📊 Data Science: Advanced spectral analysis for any time-series data"
]

for application in applications:
    # Process application
